In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import yaml

In [2]:
event_dict_file = Path("..", "resources", "data", "dictionary.yaml")
with open(event_dict_file, "r") as ed_file:
    event_dict = yaml.safe_load(ed_file)

In [3]:
print(event_dict)

{'event_type': {0: 'Announcement', 1: 'Attempt', 2: 'Corner', 3: 'Foul', 4: 'Yellow card', 5: 'Second yellow card', 6: 'Red card', 7: 'Substitution', 8: 'Free kick won', 9: 'Offside', 10: 'Hand ball', 11: 'Penalty conceded'}, 'event_type2': {12: 'Key Pass', 13: 'Failed through ball', 14: 'Sending off', 15: 'Own goal'}, 'side': {1: 'Home', 2: 'Away'}, 'shot_place': {1: 'Bit too high', 2: 'Blocked', 3: 'Bottom left corner', 4: 'Bottom right corner', 5: 'Centre of the goal', 6: 'High and wide', 7: 'Hits the bar', 8: 'Misses to the left', 9: 'Misses to the right', 10: 'Too high', 11: 'Top centre of the goal', 12: 'Top left corner', 13: 'Top right corner'}, 'shot_outcome': {1: 'On target', 2: 'Off target', 3: 'Blocked', 4: 'Hit the bar'}, 'location': {1: 'Attacking half', 2: 'Defensive half', 3: 'Centre of the box', 4: 'Left wing', 5: 'Right wing', 6: 'Difficult angle and long range', 7: 'Difficult angle on the left', 8: 'Difficult angle on the right', 9: 'Left side of the box', 10: 'Left s

In [4]:
events_file = Path("..", "resources", "data", "events.csv")
events = pd.read_csv(events_file)

In [5]:
ginf_file = Path("..", "resources", "data", "ginf.csv")
ginf = pd.read_csv(ginf_file)

In [6]:
events

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,6.0,2.0,0,9.0,2.0,1,1.0,0
1,UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
2,UFot0hit/,UFot0hit3,3,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
3,UFot0hit/,UFot0hit4,4,7,Foul by Sven Bender (Borussia Dortmund).,3,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
4,UFot0hit/,UFot0hit5,5,7,Gokhan Tore (Hamburg) wins a free kick in the ...,8,NaN,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,NaN,NaN,0,2.0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941004,z5L2OT5E/,z5L2OT5E123,123,92,Lucas Torreira (Sampdoria) wins a free kick in...,8,NaN,2,Sampdoria,Atalanta,...,NaN,NaN,NaN,NaN,0,2.0,NaN,0,NaN,0
941005,z5L2OT5E/,z5L2OT5E124,124,93,"Corner, Sampdoria. Conceded by Andrea Masiello.",2,NaN,2,Sampdoria,Atalanta,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
941006,z5L2OT5E/,z5L2OT5E125,125,93,Attempt missed. Fabio Quagliarella (Sampdoria)...,1,12.0,2,Sampdoria,Atalanta,...,NaN,NaN,8.0,2.0,0,9.0,1.0,1,3.0,0
941007,z5L2OT5E/,z5L2OT5E126,126,94,Alberto Grassi (Atalanta) wins a free kick on ...,8,NaN,1,Atalanta,Sampdoria,...,NaN,NaN,NaN,NaN,0,4.0,NaN,0,NaN,0


In [7]:
len(events["id_odsp"].unique())

9074

In [8]:
ginf

,id_odsp,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n
0,UFot0hit/,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
1,Aw5DflLH/,/soccer/germany/bundesliga-2011-2012/augsburg-...,True,2011-08-06,D1,2012,germany,FC Augsburg,SC Freiburg,2,2,2.36,3.60,3.40,NaN,NaN,NaN,NaN
2,bkjpaC6n/,/soccer/germany/bundesliga-2011-2012/werder-br...,True,2011-08-06,D1,2012,germany,Werder Bremen,Kaiserslautern,2,0,1.83,4.20,4.80,NaN,NaN,NaN,NaN
3,CzPV312a/,/soccer/france/ligue-1-2011-2012/paris-sg-lori...,True,2011-08-06,F1,2012,france,Paris Saint-Germain,Lorient,0,1,1.55,4.50,9.40,NaN,NaN,NaN,NaN
4,GUOdmtII/,/soccer/france/ligue-1-2011-2012/caen-valencie...,True,2011-08-06,F1,2012,france,Caen,Valenciennes,1,0,2.50,3.40,3.45,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10107,xAkY8l6R/,/soccer/italy/serie-a/genoa-crotone-xAkY8l6R/,True,2017-01-22,I1,2017,italy,Genoa,Crotone,2,2,1.97,4.35,8.00,1.95,2.03,2.03,1.86
10108,xSU9scI9/,/soccer/england/premier-league/chelsea-hull-ci...,True,2017-01-22,E0,2017,england,Chelsea,Hull,2,0,1.19,8.50,20.00,1.54,2.68,2.40,1.66
10109,xY7uZwOI/,/soccer/france/ligue-1/monaco-lorient-xY7uZwOI/,True,2017-01-22,F1,2017,france,AS Monaco,Lorient,4,0,1.32,6.24,11.50,1.53,3.08,1.80,2.25
10110,YyeGxMX8/,/soccer/spain/laliga/betis-gijon-YyeGxMX8/,True,2017-01-22,SP1,2017,spain,Real Betis,Sporting Gijon,0,0,1.74,4.07,5.90,2.20,1.89,2.05,1.86


In [9]:
ginf[ginf["adv_stats"] == True]

,id_odsp,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n
0,UFot0hit/,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
1,Aw5DflLH/,/soccer/germany/bundesliga-2011-2012/augsburg-...,True,2011-08-06,D1,2012,germany,FC Augsburg,SC Freiburg,2,2,2.36,3.60,3.40,NaN,NaN,NaN,NaN
2,bkjpaC6n/,/soccer/germany/bundesliga-2011-2012/werder-br...,True,2011-08-06,D1,2012,germany,Werder Bremen,Kaiserslautern,2,0,1.83,4.20,4.80,NaN,NaN,NaN,NaN
3,CzPV312a/,/soccer/france/ligue-1-2011-2012/paris-sg-lori...,True,2011-08-06,F1,2012,france,Paris Saint-Germain,Lorient,0,1,1.55,4.50,9.40,NaN,NaN,NaN,NaN
4,GUOdmtII/,/soccer/france/ligue-1-2011-2012/caen-valencie...,True,2011-08-06,F1,2012,france,Caen,Valenciennes,1,0,2.50,3.40,3.45,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10107,xAkY8l6R/,/soccer/italy/serie-a/genoa-crotone-xAkY8l6R/,True,2017-01-22,I1,2017,italy,Genoa,Crotone,2,2,1.97,4.35,8.00,1.95,2.03,2.03,1.86
10108,xSU9scI9/,/soccer/england/premier-league/chelsea-hull-ci...,True,2017-01-22,E0,2017,england,Chelsea,Hull,2,0,1.19,8.50,20.00,1.54,2.68,2.40,1.66
10109,xY7uZwOI/,/soccer/france/ligue-1/monaco-lorient-xY7uZwOI/,True,2017-01-22,F1,2017,france,AS Monaco,Lorient,4,0,1.32,6.24,11.50,1.53,3.08,1.80,2.25
10110,YyeGxMX8/,/soccer/spain/laliga/betis-gijon-YyeGxMX8/,True,2017-01-22,SP1,2017,spain,Real Betis,Sporting Gijon,0,0,1.74,4.07,5.90,2.20,1.89,2.05,1.86


In [10]:
combined_data = events.merge(ginf, on="id_odsp", suffixes=("_events", "_ginf"))

In [11]:
events.columns

Index(['id_odsp', 'id_event', 'sort_order', 'time', 'text', 'event_type',
       'event_type2', 'side', 'event_team', 'opponent', 'player', 'player2',
       'player_in', 'player_out', 'shot_place', 'shot_outcome', 'is_goal',
       'location', 'bodypart', 'assist_method', 'situation', 'fast_break'],
      dtype='object')

In [12]:
ginf.columns

Index(['id_odsp', 'link_odsp', 'adv_stats', 'date', 'league', 'season',
       'country', 'ht', 'at', 'fthg', 'ftag', 'odd_h', 'odd_d', 'odd_a',
       'odd_over', 'odd_under', 'odd_bts', 'odd_bts_n'],
      dtype='object')

In [13]:
combined_data.columns

Index(['id_odsp', 'id_event', 'sort_order', 'time', 'text', 'event_type',
       'event_type2', 'side', 'event_team', 'opponent', 'player', 'player2',
       'player_in', 'player_out', 'shot_place', 'shot_outcome', 'is_goal',
       'location', 'bodypart', 'assist_method', 'situation', 'fast_break',
       'link_odsp', 'adv_stats', 'date', 'league', 'season', 'country', 'ht',
       'at', 'fthg', 'ftag', 'odd_h', 'odd_d', 'odd_a', 'odd_over',
       'odd_under', 'odd_bts', 'odd_bts_n'],
      dtype='object')

In [14]:
pd.set_option("display.max_columns", None)

In [15]:
combined_data.head()

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,player,player2,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,mladen petric,gokhan tore,NaN,NaN,6.0,2.0,0,9.0,2.0,1,1.0,0,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
1,UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,Hamburg SV,dennis diekmeier,dennis diekmeier,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
2,UFot0hit/,UFot0hit3,3,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,NaN,1,Borussia Dortmund,Hamburg SV,heiko westermann,heiko westermann,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
3,UFot0hit/,UFot0hit4,4,7,Foul by Sven Bender (Borussia Dortmund).,3,NaN,1,Borussia Dortmund,Hamburg SV,sven bender,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
4,UFot0hit/,UFot0hit5,5,7,Gokhan Tore (Hamburg) wins a free kick in the ...,8,NaN,2,Hamburg SV,Borussia Dortmund,gokhan tore,NaN,NaN,NaN,NaN,NaN,0,2.0,NaN,0,NaN,0,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN


In [16]:
combined_data.describe(include="all")

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,player,player2,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n
count,941009,941009,941009.000000,941009.000000,941009,941009.000000,214293.000000,941009.000000,941009,941009,880009,291310,51715,51738,227459.000000,228498.000000,941009.000000,467067.000000,229185.000000,941009.000000,229137.000000,941009.000000,941009,941009,941009,941009,941009.000000,941009,941009,941009,941009.000000,941009.000000,941009.000000,941009.000000,941009.000000,98680.000000,98680.000000,98680.000000,98680.000000
unique,9074,941009,NaN,NaN,366076,NaN,NaN,NaN,142,142,6118,5747,5093,4671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9074,1,923,5,NaN,5,142,142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,ITbfCc8F/,UFot0hit1,NaN,NaN,Foul by Perparim Hetemaj (Chievo).,NaN,NaN,NaN,Juventus,Genoa,cristiano ronaldo,gonzalo castro,adrian,gonzalo castro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/soccer/england/premier-league-2015-2016/chels...,True,2015-04-04,I1,NaN,italy,Genoa,Genoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,180,1,NaN,NaN,396,NaN,NaN,NaN,12054,11981,1872,641,147,103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180,941009,3454,227127,NaN,227127,11676,11779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,53.858826,49.663663,NaN,4.326575,12.233764,1.481170,NaN,NaN,NaN,NaN,NaN,NaN,5.733693,1.926555,0.025978,6.209073,1.624831,0.264332,1.281316,0.004876,NaN,NaN,NaN,NaN,2014.405073,NaN,NaN,NaN,1.543140,1.158935,2.914901,4.243307,5.466686,2.044333,2.106117,1.938234,2.070179
std,NaN,NaN,32.014268,26.488977,NaN,2.995313,0.468850,0.499646,NaN,NaN,NaN,NaN,NaN,NaN,3.326100,0.797055,0.159071,5.421736,0.740400,0.655501,0.709394,0.069655,NaN,NaN,NaN,NaN,1.594430,NaN,NaN,NaN,1.296907,1.136129,2.325088,1.833539,5.612492,0.363461,0.531667,0.211624,0.253789
min,NaN,NaN,1.000000,0.000000,NaN,1.000000,12.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,2012.000000,NaN,NaN,NaN,0.000000,0.000000,1.060000,1.910000,1.110000,1.140000,1.420000,1.410000,1.440000
25%,NaN,NaN,27.000000,27.000000,NaN,2.000000,12.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,1.000000,0.000000,2.000000,1.000000,0.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,2013.000000,NaN,NaN,NaN,1.000000,0.000000,1.770000,3.410000,2.750000,1.790000,1.790000,1.790000,1.880000
50%,NaN,NaN,53.000000,51.000000,NaN,3.000000,12.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,2.000000,0.000000,3.000000,1.000000,0.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,2014.000000,NaN,NaN,NaN,1.000000,1.000000,2.270000,3.650000,3.860000,2.020000,1.980000,1.910000,2.050000
75%,NaN,NaN,79.000000,73.000000,NaN,8.000000,12.000000,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,9.000000,3.000000,0.000000,11.000000,2.000000,0.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,2016.000000,NaN,NaN,NaN,2.000000,2.000000,3.050000,4.300000,5.900000,2.280000,2.260000,2.050000,2.220000


In [17]:
for column_name, mapping in event_dict.items():
    print(column_name)
    mapping = {int(ev): ev_name for ev, ev_name in mapping.items()}
    print(mapping)

event_type
{0: 'Announcement', 1: 'Attempt', 2: 'Corner', 3: 'Foul', 4: 'Yellow card', 5: 'Second yellow card', 6: 'Red card', 7: 'Substitution', 8: 'Free kick won', 9: 'Offside', 10: 'Hand ball', 11: 'Penalty conceded'}
event_type2
{12: 'Key Pass', 13: 'Failed through ball', 14: 'Sending off', 15: 'Own goal'}
side
{1: 'Home', 2: 'Away'}
shot_place
{1: 'Bit too high', 2: 'Blocked', 3: 'Bottom left corner', 4: 'Bottom right corner', 5: 'Centre of the goal', 6: 'High and wide', 7: 'Hits the bar', 8: 'Misses to the left', 9: 'Misses to the right', 10: 'Too high', 11: 'Top centre of the goal', 12: 'Top left corner', 13: 'Top right corner'}
shot_outcome
{1: 'On target', 2: 'Off target', 3: 'Blocked', 4: 'Hit the bar'}
location
{1: 'Attacking half', 2: 'Defensive half', 3: 'Centre of the box', 4: 'Left wing', 5: 'Right wing', 6: 'Difficult angle and long range', 7: 'Difficult angle on the left', 8: 'Difficult angle on the right', 9: 'Left side of the box', 10: 'Left side of the six yard box'

In [18]:
for column_name, mapping in event_dict.items():
    mapping = {int(ev): ev_name for ev, ev_name in mapping.items()}
    combined_data[column_name] = combined_data[column_name].fillna(-1).astype(int)
    combined_data[column_name] = combined_data[column_name].map(mapping)
    combined_data[column_name] = combined_data[column_name].replace(-1, np.nan)

In [19]:
combined_data

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,player,player2,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,Attempt,Key Pass,Away,Hamburg SV,Borussia Dortmund,mladen petric,gokhan tore,NaN,NaN,High and wide,Off target,0,Left side of the box,left foot,Pass,Open play,0,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
1,UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis...",Corner,NaN,Home,Borussia Dortmund,Hamburg SV,dennis diekmeier,dennis diekmeier,NaN,NaN,NaN,NaN,0,NaN,NaN,None,NaN,0,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
2,UFot0hit/,UFot0hit3,3,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",Corner,NaN,Home,Borussia Dortmund,Hamburg SV,heiko westermann,heiko westermann,NaN,NaN,NaN,NaN,0,NaN,NaN,None,NaN,0,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
3,UFot0hit/,UFot0hit4,4,7,Foul by Sven Bender (Borussia Dortmund).,Foul,NaN,Home,Borussia Dortmund,Hamburg SV,sven bender,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,None,NaN,0,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
4,UFot0hit/,UFot0hit5,5,7,Gokhan Tore (Hamburg) wins a free kick in the ...,Free kick won,NaN,Away,Hamburg SV,Borussia Dortmund,gokhan tore,NaN,NaN,NaN,NaN,NaN,0,Defensive half,NaN,None,NaN,0,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941004,z5L2OT5E/,z5L2OT5E123,123,92,Lucas Torreira (Sampdoria) wins a free kick in...,Free kick won,NaN,Away,Sampdoria,Atalanta,lucas torreira,NaN,NaN,NaN,NaN,NaN,0,Defensive half,NaN,None,NaN,0,/soccer/italy/serie-a/atalanta-sampdoria-z5L2O...,True,2017-01-22,I1,2017,italy,Atalanta,Sampdoria,1,0,1.79,3.96,5.40,1.9,2.07,1.8,2.16
941005,z5L2OT5E/,z5L2OT5E124,124,93,"Corner, Sampdoria. Conceded by Andrea Masiello.",Corner,NaN,Away,Sampdoria,Atalanta,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,None,NaN,0,/soccer/italy/serie-a/atalanta-sampdoria-z5L2O...,True,2017-01-22,I1,2017,italy,Atalanta,Sampdoria,1,0,1.79,3.96,5.40,1.9,2.07,1.8,2.16
941006,z5L2OT5E/,z5L2OT5E125,125,93,Attempt missed. Fabio Quagliarella (Sampdoria)...,Attempt,Key Pass,Away,Sampdoria,Atalanta,fabio quagliarella,lucas torreira,NaN,NaN,Misses to the left,Off target,0,Left side of the box,right foot,Pass,Corner,0,/soccer/italy/serie-a/atalanta-sampdoria-z5L2O...,True,2017-01-22,I1,2017,italy,Atalanta,Sampdoria,1,0,1.79,3.96,5.40,1.9,2.07,1.8,2.16
941007,z5L2OT5E/,z5L2OT5E126,126,94,Alberto Grassi (Atalanta) wins a free kick on ...,Free kick won,NaN,Home,Atalanta,Sampdoria,alberto grassi,NaN,NaN,NaN,NaN,NaN,0,Left wing,NaN,None,NaN,0,/soccer/italy/serie-a/atalanta-sampdoria-z5L2O...,True,2017-01-22,I1,2017,italy,Atalanta,Sampdoria,1,0,1.79,3.96,5.40,1.9,2.07,1.8,2.16


In [20]:
combined_data.describe(include="all")

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,player,player2,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n
count,941009,941009,941009.000000,941009.000000,941009,941009,214293,941009,941009,941009,880009,291310,51715,51738,227459,228498,941009.000000,467067,229185,941009,229137,941009.000000,941009,941009,941009,941009,941009.000000,941009,941009,941009,941009.000000,941009.000000,941009.000000,941009.000000,941009.000000,98680.000000,98680.000000,98680.000000,98680.000000
unique,9074,941009,NaN,NaN,366076,11,4,2,142,142,6118,5747,5093,4671,13,4,NaN,19,3,5,4,NaN,9074,1,923,5,NaN,5,142,142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,ITbfCc8F/,UFot0hit1,NaN,NaN,Foul by Perparim Hetemaj (Chievo).,Free kick won,Key Pass,Home,Juventus,Genoa,cristiano ronaldo,gonzalo castro,adrian,gonzalo castro,Blocked,Off target,NaN,Defensive half,right foot,None,Open play,NaN,/soccer/england/premier-league-2015-2016/chels...,True,2015-04-04,I1,NaN,italy,Genoa,Genoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,180,1,NaN,NaN,396,237932,167859,488224,12054,11981,1872,641,147,103,54082,92827,NaN,125137,121939,773104,193747,NaN,180,941009,3454,227127,NaN,227127,11676,11779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,53.858826,49.663663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.025978,NaN,NaN,NaN,NaN,0.004876,NaN,NaN,NaN,NaN,2014.405073,NaN,NaN,NaN,1.543140,1.158935,2.914901,4.243307,5.466686,2.044333,2.106117,1.938234,2.070179
std,NaN,NaN,32.014268,26.488977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.159071,NaN,NaN,NaN,NaN,0.069655,NaN,NaN,NaN,NaN,1.594430,NaN,NaN,NaN,1.296907,1.136129,2.325088,1.833539,5.612492,0.363461,0.531667,0.211624,0.253789
min,NaN,NaN,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,2012.000000,NaN,NaN,NaN,0.000000,0.000000,1.060000,1.910000,1.110000,1.140000,1.420000,1.410000,1.440000
25%,NaN,NaN,27.000000,27.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,2013.000000,NaN,NaN,NaN,1.000000,0.000000,1.770000,3.410000,2.750000,1.790000,1.790000,1.790000,1.880000
50%,NaN,NaN,53.000000,51.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,2014.000000,NaN,NaN,NaN,1.000000,1.000000,2.270000,3.650000,3.860000,2.020000,1.980000,1.910000,2.050000
75%,NaN,NaN,79.000000,73.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,2016.000000,NaN,NaN,NaN,2.000000,2.000000,3.050000,4.300000,5.900000,2.280000,2.260000,2.050000,2.220000


In [21]:
combined_data["text"].mode()

0    Foul by Perparim Hetemaj (Chievo).
Name: text, dtype: object

In [38]:
value_counts = combined_data["text"].value_counts()
filtered_counts = value_counts[value_counts < 2]
filtered_counts

text
Attempt saved. Andreas Weimann (Aston Villa) left footed shot from outside the box is saved in the centre of the goal. Assisted by Tom Cleverley.                             1
Attempt missed. Wayne Rooney (Manchester United) right footed shot from the centre of the box is high and wide to the left. Assisted by Chris Smalling with a headed pass.    1
Attempt missed. Timo Baumgartl (VfB Stuttgart) header from very close range is high and wide to the left. Assisted by Filip Kostic.                                           1
Corner,  GFC Ajaccio. Conceded by Fabien Camus.                                                                                                                               1
Attempt missed. Franco VA¡zquez (Palermo) left footed shot from outside the box misses to the right. Assisted by Edgar Barreto.                                               1
                                                                                                                   

In [25]:
combined_data["text"].value_counts().tail(50)

text
Attempt missed. Niki Zimling (1. FSV Mainz 05) right footed shot from outside the box is too high. Assisted by Elkin Soto.                                                                     1
Attempt saved. Yunus Malli (1. FSV Mainz 05) left footed shot from the centre of the box is saved in the centre of the goal. Assisted by Sebastian Polter with a headed pass.                  1
Attempt saved. Vaclav Kadlec (Eintracht Frankfurt) right footed shot from a difficult angle on the right is saved in the centre of the goal.                                                   1
Attempt saved. Tranquillo Barnetta (Eintracht Frankfurt) right footed shot from the right side of the box is saved in the centre of the goal.                                                  1
Attempt saved. Tranquillo Barnetta (Eintracht Frankfurt) right footed shot from the right side of the box is saved in the centre of the goal. Assisted by Johannes Flum.                       1
Attempt saved. Sebastian Polte

In [51]:
value_counts = combined_data["text"].value_counts()
filtered_ixs = value_counts[value_counts < 10].index

In [52]:
filtered_combined_data = combined_data[combined_data["text"].isin(filtered_ixs)]

In [53]:
filtered_combined_data

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,player,player2,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,Attempt,Key Pass,Away,Hamburg SV,Borussia Dortmund,mladen petric,gokhan tore,NaN,NaN,High and wide,Off target,0,Left side of the box,left foot,Pass,Open play,0,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
1,UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis...",Corner,NaN,Home,Borussia Dortmund,Hamburg SV,dennis diekmeier,dennis diekmeier,NaN,NaN,NaN,NaN,0,NaN,NaN,None,NaN,0,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
5,UFot0hit/,UFot0hit6,6,9,Hand ball by Jose Paolo Guerrero (Hamburg).,Hand ball,NaN,Away,Hamburg SV,Borussia Dortmund,jose paolo guerrero,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,None,NaN,0,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
6,UFot0hit/,UFot0hit7,7,10,"Corner, Hamburg. Conceded by Lukasz Piszczek.",Corner,NaN,Away,Hamburg SV,Borussia Dortmund,lukasz piszczek,lukasz piszczek,NaN,NaN,NaN,NaN,0,NaN,NaN,None,NaN,0,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
7,UFot0hit/,UFot0hit8,8,11,Chris Lowe (Borussia Dortmund) wins a free kic...,Free kick won,NaN,Home,Borussia Dortmund,Hamburg SV,chris lowe,NaN,NaN,NaN,NaN,NaN,0,Defensive half,NaN,None,NaN,0,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941001,z5L2OT5E/,z5L2OT5E120,120,92,Alessandro Bastoni (Atalanta) wins a free kick...,Free kick won,NaN,Home,Atalanta,Sampdoria,alessandro bastoni,NaN,NaN,NaN,NaN,NaN,0,Defensive half,NaN,None,NaN,0,/soccer/italy/serie-a/atalanta-sampdoria-z5L2O...,True,2017-01-22,I1,2017,italy,Atalanta,Sampdoria,1,0,1.79,3.96,5.40,1.9,2.07,1.8,2.16
941002,z5L2OT5E/,z5L2OT5E121,121,92,Foul by Ante Budimir (Sampdoria).,Foul,NaN,Away,Sampdoria,Atalanta,ante budimir,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,None,NaN,0,/soccer/italy/serie-a/atalanta-sampdoria-z5L2O...,True,2017-01-22,I1,2017,italy,Atalanta,Sampdoria,1,0,1.79,3.96,5.40,1.9,2.07,1.8,2.16
941005,z5L2OT5E/,z5L2OT5E124,124,93,"Corner, Sampdoria. Conceded by Andrea Masiello.",Corner,NaN,Away,Sampdoria,Atalanta,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,None,NaN,0,/soccer/italy/serie-a/atalanta-sampdoria-z5L2O...,True,2017-01-22,I1,2017,italy,Atalanta,Sampdoria,1,0,1.79,3.96,5.40,1.9,2.07,1.8,2.16
941006,z5L2OT5E/,z5L2OT5E125,125,93,Attempt missed. Fabio Quagliarella (Sampdoria)...,Attempt,Key Pass,Away,Sampdoria,Atalanta,fabio quagliarella,lucas torreira,NaN,NaN,Misses to the left,Off target,0,Left side of the box,right foot,Pass,Corner,0,/soccer/italy/serie-a/atalanta-sampdoria-z5L2O...,True,2017-01-22,I1,2017,italy,Atalanta,Sampdoria,1,0,1.79,3.96,5.40,1.9,2.07,1.8,2.16


In [54]:
filtered_combined_data["text"].value_counts()

text
Offside, AtlA©tico de Madrid. Diego tries a through ball, but Falcao is caught offside.                                                                              9
Attempt blocked. Lionel Messi (Barcelona) left footed shot from outside the box is blocked. Assisted by Dani Alves.                                                  9
Alex Oxlade-Chamberlain (Arsenal) wins a free kick on the left wing.                                                                                                 9
Massimo Gobbi (Chievo) is shown the yellow card for a bad foul.                                                                                                      9
Pablo Zabaleta (Manchester City) wins a free kick in the attacking half.                                                                                             9
                                                                                                                                                                